# BERT

구글에서 개발한 자연어 처리 모델로, Transformer 구조에 기반한 사전 학습 언어 모델

핵심 개념
- Bidirectional (양방향)
  - 입력된 문장을 양방향으로 처리함. 문장의 왼쪽에서 오른쪽, 오른쪽에서 왼쪽으로 동시에 학습
  - 문맥을 더 잘 추론할 수 있게 됩
- Transformer 기반
  - Encoder 부분만을 사용한 모델.
  - 문장 내의 중요한 단어와 덜 중요한 단어를 구분하고 이를 기반으로 문장의 의미를 학습

파생 모델
- ALBERT (A Lite BERT for Self-supervised Learning of Language Representations)
- RoBERTa (A Robustly Optimized BERT Pretraining Approach)
- ELECTRA (Efficiently Learning an Encoder that Classifies Token Replacements Accurately)

## BERT를 이용한 감정 분류기 훈련

In [ ]:
!pip install datasets

모듈 준비

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import load_dataset
from torch.optim import AdamW

데이터셋 준비

In [ ]:
# 1. 데이터셋 로드
dataset = load_dataset("imdb")
train_dataset = dataset['train'].shuffle(seed=42).select(range(1000))  # 빠른 실습을 위해 1000개 샘플만 사용
test_dataset = dataset['test'].shuffle(seed=42).select(range(500))

# 2. 토크나이저 설정 (BERT-base)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 3. 토큰화 함수 정의
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

# 4. 데이터셋 토큰화
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# 5. 텐서로 변환
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8)

모델 준비

In [ ]:
# 6. BERT 모델 로드 (2개 클래스용)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# 7. 디바이스 설정 (GPU가 있으면 GPU로 설정)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 8. 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=5e-5)

훈련 함수 정의

In [ ]:
# 9. 학습 함수 정의
def train(model, dataloader):
    model.train()
    total_loss = 0
    correct = 0
    for batch in dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        _, preds = torch.max(outputs.logits, dim=1)
        correct += torch.sum(preds == labels)
    avg_loss = total_loss / len(dataloader)
    accuracy = correct.double() / len(dataloader.dataset)

    return avg_loss, accuracy

평가 함수 정의

In [ ]:
# 10. 평가 함수 정의
def evaluate(model, dataloader):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            total_loss += loss.item()
            _, preds = torch.max(outputs.logits, dim=1)
            correct += torch.sum(preds == labels)

    avg_loss = total_loss / len(dataloader)
    accuracy = correct.double() / len(dataloader.dataset)

    return avg_loss, accuracy

훈련 및 평가

In [ ]:
# 11. 학습 및 평가
epochs = 3
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    train_loss, train_acc = train(model, train_dataloader)
    print(f"Train Loss: {train_loss:.3f}, Train Accuracy: {train_acc:.3f}")

    val_loss, val_acc = evaluate(model, test_dataloader)
    print(f"Validation Loss: {val_loss:.3f}, Validation Accuracy: {val_acc:.3f}")

추론 함수 정의


In [ ]:
# 12. 추론 함수 정의 (입력된 텍스트의 감정을 예측)
def predict(text, model, tokenizer):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128).to(device)
        outputs = model(**inputs)
        _, predicted_class = torch.max(outputs.logits, dim=1)
        return predicted_class.item()

추론

In [ ]:
# 13. 예시 추론
sample_text = "I really loved this movie, it was fantastic!"
predicted_label = predict(sample_text, model, tokenizer)
print(f"입력 텍스트: \"{sample_text}\"")
print(f"예측된 레이블: {'긍정' if predicted_label == 1 else '부정'}")